**Byte-Pair Encoding**

BPE is a simple form of data compression algorithm in which the most common pair of consecutive bytes of data is replaced with a byte that does not occur in that data

**Algorithm**

1. Prepare a large enough training data (i.e. corpus)

2. Define a desired subword vocabulary size

3. Split word to sequence of characters and appending suffix “</w>” to end of word with word frequency. So the basic unit is character in this stage. For example, the frequency of “low” is 5, then we rephrase it to “l o w </w>”: 5

4. Generating a new subword according to the high frequency occurrence.

5. Repeating step 4 until reaching subword vocabulary size which is defined in step 2 or the next highest frequency pair is 1.

In [ ]:
import re, collections

In [ ]:
def get_stats(vocab):
  pairs = collections.defaultdict(int)
  for word, freq in vocab.items():
    symbols = word.split()
    for i in range(len(symbols)-1):
      pairs[symbols[i],  symbols[i+1]]+=freq
  return pairs

In [ ]:
def merge_vocab(pair, v_in):
  v_out = {}
  bigram = re.escape(" ".join(pair))
  p = re.compile(r'(?<!\S)' + bigram + r"(?!\S)")
  for word in v_in:
    w_out = p.sub("".join(pair), word)
    v_out[w_out] = v_in[word]
  return v_out

In [ ]:
vocab = {'l o w </w>' : 5, 'l o w e r </w>' : 2, 'n e w e s t </w>' : 6, 'w i d e s t </w>' : 3}

In [ ]:
num_merges = 10
for i in range(num_merges):
  pairs = get_stats(vocab)
  best = max(pairs, key = pairs.get)
  vocal = merge_vocab(best, vocab)
  print(best)



('e', 's')
('e', 's')
('e', 's')
('e', 's')
('e', 's')
('e', 's')
('e', 's')
('e', 's')
('e', 's')
('e', 's')


Note: Taking “low: 5”, “lower: 2”, “newest: 6” and “widest: 3” as an example, the highest frequency subword pair is e and s. It is because we get 6 count from newest and 3 count from widest. Then new subword (es) is formed and it will become a candidate in next iteration.

In the second iteration, the next high frequency subword pair is es (generated from previous iteration )and t. It is because we get 6count from newest and 3 count from widest.

Keep iterate until built a desire size of vocabulary size or the next highest frequency pair is 1.

**Using Library**

In [ ]:
! pip install bpe


     |████████████████████████████████| 376 kB 5.6 MB/s 
     |████████████████████████████████| 14.0 MB 47.6 MB/s 
     |████████████████████████████████| 843 kB 46.4 MB/s 


In [ ]:
from bpe import Encoder

In [ ]:


# Generated with http://pythonpsum.com
test_corpus = '''
    Object raspberrypi functools dict kwargs. Gevent raspberrypi functools. Dunder raspberrypi decorator dict didn't lambda zip import pyramid, she lambda iterate?
    Kwargs raspberrypi diversity unit object gevent. Import fall integration decorator unit django yield functools twisted. Dunder integration decorator he she future. Python raspberrypi community pypy. Kwargs integration beautiful test reduce gil python closure. Gevent he integration generator fall test kwargs raise didn't visor he itertools...
    Reduce integration coroutine bdfl he python. Cython didn't integration while beautiful list python didn't nit!
    Object fall diversity 2to3 dunder script. Python fall for: integration exception dict kwargs dunder pycon. Import raspberrypi beautiful test import six web. Future integration mercurial self script web. Return raspberrypi community test she stable.
    Django raspberrypi mercurial unit import yield raspberrypi visual rocksdahouse. Dunder raspberrypi mercurial list reduce class test scipy helmet zip?
'''

encoder = Encoder(200, pct_bpe=0.88)  # params chosen for demonstration purposes
encoder.fit(test_corpus.split('\n'))

example = "Vizzini: He didn't fall? INCONCEIVABLE!"
print(encoder.tokenize(example))
print()
# ['__sow', 'vi', 'z', 'zi', 'ni', '__eow', '__sow', ':', '__eow', 'he', 'didn', "'", 't', 'fall', '__sow', '?', '__eow', '__sow', 'in', 'co', 'n', 'ce', 'iv', 'ab', 'le', '__eow', '__sow', '!', '__eow']
print(next(encoder.transform([example])))
print()
# [24, 108, 82, 83, 71, 25, 24, 154, 25, 14, 10, 11, 12, 13, 24, 85, 25, 24, 140, 59, 39, 157, 87, 165, 114, 25, 24, 148, 25]
print(next(encoder.inverse_transform(encoder.transform([example]))))
# vizzini : he didn ' t fall ? inconceivable !

['__sow', 'vi', 'z', 'zi', 'ni', '__eow', '__sow', ':', '__eow', 'he', 'didn', "'", 't', 'fall', '__sow', '?', '__eow', '__sow', 'in', 'co', 'n', 'ce', 'iv', 'ab', 'le', '__eow', '__sow', '!', '__eow']

[25, 108, 82, 83, 71, 24, 25, 154, 24, 14, 10, 11, 12, 13, 25, 85, 24, 25, 140, 59, 39, 157, 87, 165, 114, 24, 25, 148, 24]

vizzini : he didn ' t fall ? inconceivable !
